# Machine Learning - Practical 4

Names: {YOUR NAMES}

In [1]:
import numpy as np
import cvxopt
from matplotlib import pyplot as plt
from sklearn.svm import SVC
from sklearn.datasets import load_wine
from sklearn.tree import DecisionTreeClassifier

## Task 0: The Data

We will work with the data from Practical 3. Load the data and split it into a training and test set. You can re-use the data splitting function from Practical 2.

In [2]:
# Split data in training and test set
def split_data(X, y, frac=0.3, seed=None):
    if seed is not None:
        np.random.seed(seed)

    # ---------------- INSERT CODE ----------------------
    idx = X.shape[0]

    n_trainset = int(idx*(1-frac))  # size of the training set
    n_testset = idx-n_trainset  # size of the test set

    idx_shuffled = np.random.permutation(idx)

    test_idx = idx_shuffled[:n_testset]
    train_idx = idx_shuffled[n_testset:n_testset+n_trainset]

    X_test = X[test_idx]
    y_test = y[test_idx]
    print('Test set shapes (X and y)', X_test.shape, y_test.shape)

    X_train = X[train_idx]
    y_train = y[train_idx]
    print('Training set shapes (X and y):', X_train.shape, y_train.shape)
    

    # ---------------- END CODE -------------------------
    
    return X_train, X_test, y_train, y_test


In [3]:
# load data
X_2d, t_2d = np.load('nonlin_2d_data.npy')[:,:2], np.load('nonlin_2d_data.npy')[:, 2]

In [4]:
# split data
X_train, X_test, t_train, t_test = split_data(X_2d, t_2d, seed=1)

Test set shapes (X and y) (75, 2) (75,)
Training set shapes (X and y): (175, 2) (175,)


## Task 1: Support Vector Machines

First, you will implement a training algorithm for the Support Vector Machine (SVM). For solving the quadratic program, we provide a simple interface to the cvxopt library below.

In SVMs, each data sample $x_n$ has a corresponding lagrange multiplier $\alpha_n$ which indicates if $x_n$ is a support vector. In the latter case $\alpha_n > 0$ holds. 
The goal of learning the SVM is to figure out which samples are support vectors by learning $\mathbf{\alpha}$. The dual SVM optimizes the following quadratic program.

$$ \min \frac{1}{2} \sum_{n=1}^N \sum_{m=1}^N \alpha_n \alpha_m t_n t_m k(\mathbf{x}_n, \mathbf{x}_m) - \sum_{n=1}^N \alpha_n$$
subject to 
$$ 0 \leq \alpha_n \leq C $$
$$ \sum_{n=1}^N \alpha_n t_n = 0 $$ 

The quadratic program solver expects the following form:
$$ \min \frac{1}{2} \alpha^T P \alpha + \mathbf q^T \mathbf \alpha $$
subject to 

$$A \alpha = b$$
$$G \alpha \leq h $$

Here, $A$ and $G$ are matrices with one row per individual constraint. Similarly, $b$ (the intercept or bias) and $h$ are vectors with one element per individual constraint.

Having trained the SVM, a prediction for an input $\mathbf{x}$ is made by:

$$ y = sign([\sum_n^{N} \alpha_n t_n k(\mathbf{x}, \mathbf{x}_n)] + b)  $$


### Task 1.1
 
Use the code provided below as a basis to express the constrained optimization problem in terms of $P, q, A, b, G$ and $h$ and implement a function `fit_svm` which passes these variables to the provided QP solver. Fit a SVM on the training data and extract its parameters using a linear kernel (dot product).

**Hints:**
  - The box constraint $0 \leq \alpha_n \leq C$ defines two constraints of the form $G \alpha_n \leq h$ for each $\alpha_n$.
  - The inequality $x \geq 0$ is equivalent to $-x \leq 0$.
  - The SVM is described in chapter 12 of Elements of Statistical Learning.

In [75]:
def linear_kernel(a, b):
    # ---------------- INSERT CODE ----------------------
    return a @ b
    # ---------------- END CODE -------------------------
    
def fit_svm(X, t, kernel, C=1.0):
    '''Fit SVM using data (X,t), specified kernel and parameter C.
    Inputs
        X:  predictors
        t:  targets
        C:  constant
    '''

    t = np.array([-1. if l == 0 else 1. for l in t])
    # ---------------- INSERT CODE ----------------------
    N = len(t)
    P = np.zeros((len(X),len(X)))
    for i in range(N):
        for j in range(N):
            P[i][j]=t[i]*t[j]*kernel(X[i],X[j])
    q = -1*np.ones(N)
    A = t.reshape(1,N)
    print(np.shape(A))
    b = np.zeros(1).reshape(1,1)
    G = np.zeros(2*N*N)
    G = G.reshape(2*N,N)
    for i in range(N):
         G[i,i]=1
    h = np.array([C for i in range(N)])
    h = np.append(h,np.zeros(N))     
    # ---------------- END CODE -------------------------
    
    assert P.shape == (len(X), len(X))
    assert len(q) == len(X)
    assert A.shape == (1, len(t)) and A.dtype == 'float'
    assert b.shape == (1, 1)
    assert len(G) == 2 * len(X)
    assert len(h) == 2 * len(X)

    return solve_quadratic_program(P, q, A, b, G, h)

def solve_quadratic_program(P, q, A, b, G, h):
    '''Uses cvxopt to solve the quadratic program.'''
    P, q, A, b, G, h = [cvxopt.matrix(var) for var in [P, q, A, b, G, h]]
    minimization = cvxopt.solvers.qp(P, q, G, h, A, b)
    lagr_mult = np.ravel(minimization['x'])
    return lagr_mult


def extract_parameters(X, t, kernel, lagr_mult, threshold=1e-7):
    '''Computes the intercept from the support vector constraints.
    
    Inputs
        X:         predictors
        t:         targets
        kernel:    a kernel to be used
        lagr_mult: the Lagrange multipliers obtained by solving the dual QP
        threshold: threshold for choosing support vectors
    
    Returns
        lagr_mult: lagrange multipliers for the support vectors
        svs:       set of support vectors
        sv_labels: targets t_n for the support vectors
        intercept: computed intercept (also called bias)
    '''
    t = np.array([-1. if l == 0 else 1. for l in t])
    
    # ---------------- INSERT CODE ----------------------
    A = t.reshape((1,len(t)))
    intercept = A @ lagr_mult
    svs_indices = np.where(lagr_mult < threshold)
    svs = X[svs_indices]
    sv_labels = np.sign(svs)
    lagr_mult = lagr_mult[svs_indices]
    # ---------------- END CODE -------------------------
    

    return lagr_mult, svs, sv_labels, intercept


In [76]:
# Training
# Fit SVM on training data
lagrange_multiplier = fit_svm(X_train,t_train, linear_kernel)

# Extract parameters
lagr_mult , svs,sv_labels, intercept = extract_parameters(X_train, t_train, linear_kernel, lagrange_multiplier)

print(svs.shape,sv_labels.shape, intercept)


(1, 124)
     pcost       dcost       gap    pres   dres
 0: -2.7068e+01 -2.4223e+02  2e+03  3e+00  4e+00
 1: -1.7354e+00 -1.4036e+02  1e+02  3e-02  4e-02
 2: -1.6271e+01 -5.6256e+01  4e+01  7e-03  9e-03
 3: -3.2577e+01 -4.9012e+01  2e+01  2e-03  3e-03
 4: -3.8620e+01 -4.6233e+01  9e+00  8e-04  1e-03
 5: -4.2191e+01 -4.4319e+01  3e+00  2e-04  3e-04
 6: -4.3305e+01 -4.3923e+01  7e-01  4e-05  5e-05
 7: -4.3656e+01 -4.3816e+01  2e-01  9e-06  1e-05
 8: -4.3770e+01 -4.3778e+01  9e-03  4e-07  5e-07
 9: -4.3776e+01 -4.3776e+01  1e-04  4e-09  7e-09
10: -4.3776e+01 -4.3776e+01  1e-06  4e-11  4e-09
Optimal solution found.
(12, 13) (12, 13) [-3.55271368e-15]


### Task 1.2

Having learnt an SVM, we can use the calculated parameters to make predictions on novel samples.
- Implement a function `svm_predict(X, kernel, lagr_mult, svs, sv_labels, intercept)`.
- Use this function with the linear kernel and compute the test accuracy on the 2d dataset.
- Visualize the samples form the test set in a scatter plot colored by your predictions

In [85]:
def svm_predict(X, kernel, lagr_mult, svs, sv_labels, intercept):
    ''' Given the learned parameters of the SVM, make a prediction on the test set.
    Inputs
        X:         predictors
        kernel:    a kernel to be used
        lagr_mult: the Lagrange multipliers obtained by solving the dual QP
        svs:       set of support vectors
        sv_labels: targets t_n for the support vectors
        intercept: computed intercept (also called bias)
    
    Returns
        prediction: predictions on novel samples
    '''  
    
    # ---------------- INSERT CODE ----------------------
    K = np.array([kernel(sv , xv) for sv in svs for xv in X ]).reshape(svs.shape[0],X.shape[0])
    print(K)
    print(svs.shape)
    prediction = [(alpha_n * t_n) * K for alpha_n,t_n in zip(lagr_mult,sv_labels)] 
    # ---------------- END CODE -------------------------
    
    return prediction

In [86]:
# Testing
# make predictions for test set
test_predictions = svm_predict(X_test,linear_kernel,lagr_mult, svs, sv_labels, intercept)

[[2129072.2259     1069106.6614      819162.2308      840816.6742
   706433.5582     1766604.7593     1666268.35       2548733.9991
  1749713.0567     1254090.6177     1430563.4732     2189244.0056
  2213479.6425      476870.0722     1942433.1073     1592209.1598
  1324567.0563     2448653.1126     1059921.2345     2557259.5068
  1153114.2592     1138718.0134     1002759.0307     1826992.2539
  1911447.7231     1894756.9573     1086920.9107      707028.2868
   747009.7622     2171040.5569      692122.1797     1248160.0588
  1228692.8257     1674984.0523     2178101.101       815676.3301
  1071205.6991     1424784.097      1499972.2956      884720.4752
  1209371.8206     1203800.882       852615.2455     1219792.1891
  1150176.8104     1852061.2824      728721.6421      952035.3447
   829523.4567      634103.9627     1408465.4175     1153801.0749
   865798.3032     1102581.1385    ]
 [ 720246.8298      364666.6691      282382.6671      287338.3604
   242386.8793      598169.6837      56

ValueError: operands could not be broadcast together with shapes (13,) (12,54) 

In [9]:
# visualize predictions


### Task 1.3

- Instead of using the linear kernel, use the Gaussian RBF kernel defined in Practical 3.
- Compare results on with both kernels with sklearn implementation (SVC)
- Visualize the predictions on the test set, the learned support vectors and the decision boundary for both kernels (Hint: Adapt the decision boundary plot from Practical 3).

In [10]:
def rbf_kernel(a, b):
    # ---------------- INSERT CODE ----------------------
    return np.exp(np.sqrt((a-b)**2))


    # ---------------- END CODE -------------------------

In [11]:
# Fit SVM with rbf kernel and calculate the test accuracy


In [12]:
# Fit SVM using sklearn and calculate the test accuracy


In [13]:
# Visualize


# Task 2: Decision Trees

Next, we will implement a simple decision tree classifier using the Wine dataset, one of the standard sklearn datasets. 

We will use the Gini impurity as a criterion for splitting. It is defined for a set of labels as
$$ G = \sum_{i=0}^C p(i) * (1- p(i)) $$

Given labels $l$ and split $l_a$ and $l_b$, the weighted removed impurity can be computed by $G(l) - \frac{|l_a|}{|l|}G(l_a) - \frac{|l_b|}{|l|}G(l_b)$.

Here is a simple explanation of the Gini impurity that you may find useful: https://victorzhou.com/blog/gini-impurity/


### Task 2.1

1. Plot the distribution of the first feature of for each class of the wine dataset.
2. Implement a function `gini_impurity(t)` that computes the Gini impurity for an array of labels `t`.
3. Calculate the removed Gini impurity for a split after 50 samples, i.e. between `t[:50]` and `t[50:]`.

In [14]:
# Load Wine dataset and split into train+test set

X, t = load_wine(return_X_y=True)
X_train, X_test, t_train, t_test = split_data(X, t)

Test set shapes (X and y) (54, 13) (54,)
Training set shapes (X and y): (124, 13) (124,)


In [15]:
# Plot distribution


In [16]:
# Compute Gini impurity


### Task 2.2
For each of the first 12 features, compute the remove Gini impurity for every possible split. Visualize the removed Gini impurity per feature across all splits. Which is the optimal split?

In [17]:
# Plotting


### Task 2.3

1. Implement a function `build_tree(X, t, depth)` which recursively builds a tree. Use the classes `Node` and `Leaf` as a data structure to build your tree.
2. Implement a function `predict_tree(tree, x)` which makes a prediction for sample `x`. Obtain scores for the `wine` dataset and compare to `sklearn.tree.DecisionTree`.
3. Switch back to the synthetic 2d dataset from the beginning (kernel methods). Compute scores and visualize the decisions in a 2d grid.

In [18]:
class Node:
    def __init__(self, left, right, n_feat, threshold):
        self.left = left
        self.right = right
        self.n_feat = n_feat
        self.threshold = threshold


class Leaf:
    def __init__(self, label):
        self.label = label


In [19]:
# Implement recursive tree function

def build_tree(X, t, depth, max_depth=3, n_labels=2):
    
    # ---------------- INSERT CODE ----------------------



    # ---------------- END CODE -------------------------

    
def predict_tree(node, x):
    
    # ---------------- INSERT CODE ----------------------



    # ---------------- END CODE -------------------------

IndentationError: expected an indented block (3238061611.py, line 12)

In [ ]:
# Build tree

tree = build_tree(X_train, t_train, 0, max_depth=3, n_labels=3)

In [ ]:
# Calculate training and test scores


In [ ]:
# Calculate test score using sklearn


In [ ]:
# Calculate test score for synthetic 2D dataset


In [ ]:
# Visualize
